In [1]:
PATH2LDM="/project_ghent/Mostafa/image/ImageTransmission/src/gen_comm/vqgan_lc/VQGAN-LC/ldm-lc"
PATH2CAI='/project_ghent/Mostafa/image/compress_AI'
PATH2Progdtd='/project_ghent/Mostafa/image/ImageTransmission/src/gen_comm/ProgDTD'

PATH2KODAK = "/project_ghent/Mostafa/image/kodak/kodim"

PATH2HYPPROG = '/project_ghent/Mostafa/image/ImageTransmission/src/gen_comm/ProgDTD/ckpts/Lambda=0.1 - range=[0.0-1.0].zip'

PATH2CH = '/project_ghent/Mostafa/image/ImageTransmission/Paper/tx_latency/channel_rates(-5dB).npy'
########################
import sys
sys.path.append(PATH2LDM)
sys.path.append(PATH2CAI)
sys.path.append(PATH2Progdtd)

import torch
from PIL import Image
import os
import numpy as np
import pandas as pd

from utils import compute_metrics

from compressai.ans import BufferedRansEncoder, RansDecoder

/project_ghent/Mostafa/image/compress_AI/compressai/models/video/google.py:353: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @amp.autocast(enabled=False)


In [2]:
import pkg_resources, subprocess


def install_if_needed(package, version):
    try:
        pkg_resources.require(f"{package}=={version}")
    except:
        subprocess.check_call([f"pip", "install", f"{package}=={version}"])


install_if_needed("pip", "23.3.1")
install_if_needed("pytorch_lightning", "1.7.7")
install_if_needed("torchmetrics", "0.11.0")


/tmp/ipykernel_14975/1776380054.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources, subprocess


DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def get_image_as_tensor(number):
    return torch.tensor(
        np.array(Image.open(f"{PATH2KODAK}{str(number % 23 + 1).zfill(2)}.png").convert('RGB'))
    ).permute(2, 0, 1).float().cuda().unsqueeze(0) / 255.0


In [5]:
pl_model = torch.load(PATH2HYPPROG, map_location=torch.device(device))
model = pl_model.model
model.encoder = BufferedRansEncoder()
model.decoder = RansDecoder()

img = get_image_as_tensor(0)
model.update();
model.eval();

/tmp/ipykernel_14975/2979685059.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_model = torch.load(PATH2HYPPROG, map_location=torch.device(device))


In [6]:
MAX_FEATURES = 32  #  8 12 24 32 48 96 192 Total number of feature maps per image

NUM_IMAGES = 24

# Parameters
IMAGE_INDICES = list(range(NUM_IMAGES))  # image_idx from 0 to 23
CHANNEL_CODING_RATE = 2/3

def initialize_image_states():
    """
    Initializes the state for each image.
    """
    image_states = {}
    for img_idx in IMAGE_INDICES:
        # img = torch.tensor(kodak.generate_image(image_idx=img_idx)).cuda() / 255.0
        image_states[img_idx] = {
            "feature_0": 0,  # Index of the first feature map to transmit
            "remaining_features": MAX_FEATURES,  # Number of feature maps left to transmit
            "completed": False,  # Flag indicating if all features have been transmitted
            "start_slot": None,  # Slot when image transmission started
            "end_slot": None,  # Slot when image transmission completed
            "latency": None,  # To store waiting time once transmission is complete
            "total_bpp": 0.0,  # Cumulative BPP across multiple transmissions
        }
    return image_states

# Initialize image states
image_states = initialize_image_states()
image_queue = list(IMAGE_INDICES)  # Queue to manage image transmission order

# Variables to store past feature maps and scales for the current image
current_past_feature_maps = None
current_past_scales = None

# Initialize lists to store metrics per slot
slot_psnr = []
slot_ssim = []
slot_bpp = []
slot_channel_rate = []  # To store channel rates per slot
slot_indices = []
slot_images_transmitted = []
slot_latency = []
img_waiting = []
img_waiting_idx = []


channel_rates = CHANNEL_CODING_RATE * np.load(PATH2CH)

# Initialize current image index
current_image_idx = image_queue[0] if image_queue else None

for slot_i, rate in enumerate(channel_rates):
    print(f"\n--- Slot {slot_i + 1} ---")
    print(f"Channel Rate: {rate} bits")
    remaining_bits = rate  # Initialize remaining bits for this slot

    # Initialize temporary lists to collect metrics for this slot
    current_slot_psnr = []
    current_slot_ssim = []
    current_slot_bpp = []
    current_slot_waiting_times = []
    current_slot_latency = []
    current_slot_images_transmitted = 0

    transmission_occurred = False  # Flag to check if any transmission occurred in this slot

    while remaining_bits > 0 and current_image_idx is not None:
        state = image_states[current_image_idx]
        state["img"] = get_image_as_tensor(current_image_idx)

        # If transmission hasn't started, set the start_slot and reset total_bpp
        if state['start_slot'] is None:
            state['start_slot'] = slot_i
            state['total_bpp'] = 0.0  # Reset cumulative BPP for new transmission

        feature_0 = state["feature_0"]
        remaining_features = state["remaining_features"]

        # Initialize N_features to 1 and find the maximum N_features that fits within remaining_bits
        N_features = 1
        best_N_features = 0
        best_compressed_prog = None
        best_nbits = 0

        while N_features <= remaining_features:
            with torch.no_grad():
                compressed_prog = model.my_compress(
                    img=state["img"],
                    feature_0=feature_0,
                    N_feature=N_features
                )
                # Calculate the number of bits: assuming 'y_string' and 'z_string' are byte sequences
                nbits = 8 * (len(compressed_prog['y_string']) + len(compressed_prog['z_string']))

            if nbits <= remaining_bits:
                best_N_features = N_features
                best_compressed_prog = compressed_prog
                best_nbits = nbits
                N_features += 1  # Try to send one more feature map
            else:
                break  # Exceeded the bit budget

        if best_N_features > 0:
            # Transmit the best_N_features for this image
            img_hat, past_feature_maps, past_scales = model.my_decompress(
                best_compressed_prog,
                feature_0=feature_0,
                N_feature=best_N_features,
                y_hat_old=current_past_feature_maps,
                scales_old=current_past_scales
            )
            # Update metrics
            metrics = compute_metrics(state["img"], img_hat)  # Ensure this returns {'psnr': value, 'ssim': value}
            npixels = state["img"].shape[-1] * state["img"].shape[-2]
            bpp_value = best_nbits / npixels

            # Accumulate BPP for the image
            state["total_bpp"] += bpp_value

            print(f"Image {current_image_idx}: Transmitted {best_N_features} features @ Cumulative BPP: {state['total_bpp']:.4f}, PSNR: {metrics['psnr']:.2f}, SSIM: {metrics['ssim']:.4f}")

            # Append metrics to current slot's lists
            current_slot_psnr.append(metrics['psnr'])
            current_slot_ssim.append(metrics['ssim'])
            current_slot_bpp.append(state["total_bpp"])  # Use cumulative BPP

            # Update transmission state
            state["feature_0"] += best_N_features
            state["remaining_features"] -= best_N_features
            remaining_bits -= best_nbits
            transmission_occurred = True

            # Update past feature maps and scales
            current_past_feature_maps = past_feature_maps
            current_past_scales = past_scales

            # Check if all features have been transmitted
            if state["remaining_features"] <= 0:
                state["completed"] = True
                state['end_slot'] = slot_i
                waiting_time = state['end_slot'] - state['start_slot'] + 1
                img_waiting.append(waiting_time)
                img_waiting_idx.append(slot_i)
                state['latency'] = waiting_time
                current_slot_images_transmitted += 1
                print(f"Image {current_image_idx}: All feature maps transmitted in {waiting_time} slots.")
                current_slot_waiting_times.append(waiting_time)
                current_slot_latency.append(state['latency'])

                # Reset the image state for the next transmission round
                state["feature_0"] = 0
                state["remaining_features"] = MAX_FEATURES
                state["completed"] = False
                state["start_slot"] = None
                state["end_slot"] = None
                state["waiting_time"] = None

                # Reset past feature maps and scales
                current_past_feature_maps = None
                current_past_scales = None

                # Move the image to the end of the queue for the next round
                image_queue.append(image_queue.pop(0))

                # Update the current_image_idx to the next image in the queue
                current_image_idx = image_queue[0] if image_queue else None
            else:
                # Continue transmitting the same image in the next iteration if possible
                pass
        else:
            print(f"Image {current_image_idx}: Unable to transmit any feature maps in this slot due to insufficient bits.")
            # Move to the next slot without attempting further transmissions in this slot
            break  # Exit the while loop to move to the next slot

    # After processing all possible transmissions in this slot, store the metrics
    # Compute average metrics for this slot
    avg_psnr = np.mean(current_slot_psnr) if current_slot_psnr else 0
    avg_ssim = np.mean(current_slot_ssim) if current_slot_ssim else 0
    avg_bpp = np.mean(current_slot_bpp) if current_slot_bpp else 0
    # avg_waiting_time = np.mean(current_slot_waiting_times) if current_slot_waiting_times else 0
    avg_latency = np.mean(current_slot_latency) if current_slot_latency else 0


    # Append the channel rate for this slot
    slot_channel_rate.append(rate)

    # Append averages to the slot metrics lists
    slot_psnr.append(avg_psnr)
    slot_ssim.append(avg_ssim)
    slot_bpp.append(avg_bpp)
    # slot_waiting_time.append(avg_waiting_time)
    slot_images_transmitted.append(current_slot_images_transmitted)
    slot_indices.append(slot_i + 1)  # Slot indices start from 1
    slot_latency.append(avg_latency)




# After all slots, print a summary if needed
print("\n--- Transmission Complete ---\n")

# Define the number of slots
num_slots = len(channel_rates)
slots = slot_indices

if state["remaining_features"] < MAX_FEATURES and state['start_slot'] is not None:
    waiting_time = num_slots - state['start_slot'] + 1
    img_waiting.append(waiting_time)
    img_waiting_idx.append(num_slots)
    
# Optionally, save all metrics to a CSV file for further analysis
metrics_df = pd.DataFrame({
    'Slot': slots,
    'Channel_Rate_bits': slot_channel_rate,
    'Average_PSNR_dB': slot_psnr,
    'Average_SSIM': slot_ssim,
    'Average_BPP_bits_per_pixel': slot_bpp,
    'Average_Latency_Slots': slot_latency,
    'Images_Transmitted': slot_images_transmitted
})

metrics_df2 = pd.DataFrame({
    'img_waiting': img_waiting,
    'img_waiting_idx': img_waiting_idx
})



--- Slot 1 ---
Channel Rate: 27304.776929021366 bits
Image 0: Transmitted 2 features @ Cumulative BPP: 0.0580, PSNR: 19.55, SSIM: 0.2957
Image 0: Unable to transmit any feature maps in this slot due to insufficient bits.

--- Slot 2 ---
Channel Rate: 26413.559164969196 bits
Image 0: Transmitted 3 features @ Cumulative BPP: 0.1171, PSNR: 20.74, SSIM: 0.3690
Image 0: Unable to transmit any feature maps in this slot due to insufficient bits.

--- Slot 3 ---
Channel Rate: 25616.271400978254 bits
Image 0: Transmitted 3 features @ Cumulative BPP: 0.1725, PSNR: 21.73, SSIM: 0.4502
Image 0: Unable to transmit any feature maps in this slot due to insufficient bits.

--- Slot 4 ---
Channel Rate: 24925.93377986572 bits
Image 0: Transmitted 3 features @ Cumulative BPP: 0.2226, PSNR: 22.30, SSIM: 0.5004
Image 0: Unable to transmit any feature maps in this slot due to insufficient bits.

--- Slot 5 ---
Channel Rate: 24354.05680177223 bits
Image 0: Transmitted 3 features @ Cumulative BPP: 0.2716, PS